<a href="https://colab.research.google.com/github/DHjeelm/OpenProjectAboutGANs/blob/main/CycleGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-rv6yxn7o
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-rv6yxn7o
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101077 sha256=388b2c737e775e8ab768f2de5f3b90642c0e9e213e6e8ab1f157a45355be097c
  Stored in directory: /tmp/pip-ephem-wheel-cache-5lkfphyd/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Imports
import tensorflow as tf
from random import random
from numpy import load
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randint
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from matplotlib import pyplot

In [ ]:
# Define discriminnator
def define_discriminator(image_shape):

  # Weight initialization
  init = RandomNormal(stddev=0.02)

  # Input
  in_image = Input(shape=image_shape)

  # Architecture
  d = Conv2D(64, (4,4), strides=(2,2), padding="same", kernel_initializer=init)(in_image)
  d = LeakyReLU(alpha=0.2)(d)
  d = Conv2D(128, (4,4), strides=(2,2), padding="same", kernel_initializer=init)(d)
  d = InstanceNormalization(axis=-1)(d)
  d = LeakyReLU(alpha=0.2)(d)
  d = Conv2D(256, (4,4), strides=(2,2), padding="same", kernel_initializer=init)(d)
  d = InstanceNormalization(axis=-1)(d)
  d = LeakyReLU(alpha=0.2)(d)
  d = Conv2D(512, (4,4), strides=(2,2), padding="same", kernel_initializer=init)(d)
  d = InstanceNormalization(axis=-1)(d)
  d = LeakyReLU(alpha=0.2)(d)
  d = Conv2D(512, (4,4), padding="same", kernel_initializer=init)(d)
  d = InstanceNormalization(axis=-1)(d)
  d = LeakyReLU(alpha=0.2)(d)
  patch_out = Conv2D(1, (4,4), padding="same", kernel_initializer=init)(d)

  # Create and compile model
  model = Model(in_image, patch_out)
  model.compile(loss="mse", optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5), loss_weights=[0.5])
  return model

In [ ]:
# Resnet block for generator
def resnet_block(n_filters, input_layer):
  # weight initialization
  init = RandomNormal(stddev=0.02)
  
  # Architecture
  g = Conv2D(n_filters, (3,3), padding="same", kernel_initializer=init)(input_layer)
  g = InstanceNormalization(axis=-1)(g)
  g = Activation("relu")(g)
  g = Conv2D(n_filters, (3,3), padding="same", kernel_initializer=init)(g)
  g = InstanceNormalization(axis=-1)(g)
  g = Concatenate()([g, input_layer])
  return g

In [ ]:
# define the standalone generator model
def define_generator(image_shape, n_resnet=9):

  # Weight initialization
  init = RandomNormal(stddev=0.02)

  # Input
  in_image = Input(shape=image_shape)

  # Architecture
  g = Conv2D(64, (7,7), padding="same", kernel_initializer=init)(in_image)
  g = InstanceNormalization(axis=-1)(g)
  g = Activation("relu")(g)
  g = Conv2D(128, (3,3), strides=(2,2), padding="same", kernel_initializer=init)(g)
  g = InstanceNormalization(axis=-1)(g)
  g = Activation("relu")(g)
  g = Conv2D(256, (3,3), strides=(2,2), padding="same", kernel_initializer=init)(g)
  g = InstanceNormalization(axis=-1)(g)
  g = Activation("relu")(g)
  for _ in range(n_resnet):
    g = resnet_block(256, g)
  g = Conv2DTranspose(128, (3,3), strides=(2,2), padding="same", kernel_initializer=init)(g)
  g = InstanceNormalization(axis=-1)(g)
  g = Activation("relu")(g)
  g = Conv2DTranspose(64, (3,3), strides=(2,2), padding="same", kernel_initializer=init)(g)
  g = InstanceNormalization(axis=-1)(g)
  g = Activation("relu")(g)
  g = Conv2D(3, (7,7), padding="same", kernel_initializer=init)(g)
  g = InstanceNormalization(axis=-1)(g)
  out_image = Activation("tanh")(g)

  # Define model
  model = Model(in_image, out_image)
  return model

In [ ]:
# Define a composite model for updating generators by adversarial and cycle loss

def define_composite_model(g_model_1, d_model, g_model_2, image_shape):

  # ensure the model we✬re updating is trainable
  g_model_1.trainable = True

  # mark discriminator as not trainable
  d_model.trainable = False

  # mark other generator model as not trainable
  g_model_2.trainable = False

  # Discriminator element
  input_gen = Input(shape=image_shape)
  gen1_out = g_model_1(input_gen)
  output_d = d_model(gen1_out)

  # Identity element
  input_id = Input(shape=image_shape)
  output_id = g_model_1(input_id)

  # Forward cycle
  output_f = g_model_2(gen1_out)

  # Backward cycle
  gen2_out = g_model_2(input_id)
  output_b = g_model_1(gen2_out)

  # Define model graph
  model = Model([input_gen, input_id], [output_d, output_id, output_f, output_b])

  # Define optimization algorithm configuration
  opt = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

  # Compile model with weighting of least squares loss and L1 loss
  model.compile(loss=["mse", "mae", "mae", "mae"], loss_weights=[1, 5, 10, 10],
  optimizer=opt)
  return model

In [ ]:
# Load and prepare training images
def load_real_samples(filename):

  # Load the dataset
  data = load(filename)

  # Unpack arrays
  X1, X2 = data["arr_0"], data["arr_1"]

  # Scale from [0,255] to [-1,1]
  X1 = (X1 - 127.5) / 127.5
  X2 = (X2 - 127.5) / 127.5

  return [X1, X2]

In [ ]:
# Select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples, patch_shape):

  # Random instances
  ix = randint(0, dataset.shape[0], n_samples)

  # Retrieve selected images
  X = dataset[ix]

  # Generate ✬real✬ class labels (1)
  y = ones((n_samples, patch_shape, patch_shape, 1))
  return X, y

In [ ]:
# Generate a batch of images, returns images and targets
def generate_fake_samples(g_model, dataset, patch_shape):

  # Generate fake instance
  X = g_model.predict(dataset)

  # Create "fake" class labels (0)
  y = zeros((len(X), patch_shape, patch_shape, 1))
  return X, y

In [ ]:
# Update image pool for fake images
def update_image_pool(pool, images, max_size=50):
  selected = list()
  for image in images:

    if len(pool) < max_size:
    # Stock the pool
      pool.append(image)
      selected.append(image)

    elif random() < 0.5:
      # Use image, but don✬t add it to the pool
      selected.append(image)

    else:
      # Replace an existing image and use replaced image
      ix = randint(0, len(pool))
      selected.append(pool[ix])
      pool[ix] = image
      
  return asarray(selected)

In [ ]:
# save the generator models to file
def save_models(step, g_model_AtoB, g_model_BtoA):
  # save the first generator model
  filename1 = "g_model_AtoB_%06d.h5" % (step+1)
  g_model_AtoB.save(filename1)
  # save the second generator model
  filename2 = "g_model_BtoA_%06d.h5" % (step+1)
  g_model_BtoA.save(filename2)
  print(">Saved: %s and %s" % (filename1, filename2))

In [ ]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, trainX, name, n_samples=5):
  # select a sample of input images
  X_in, _ = generate_real_samples(trainX, n_samples, 0)
  # generate translated images
  X_out, _ = generate_fake_samples(g_model, X_in, 0)
  # scale all pixels from [-1,1] to [0,1]
  X_in = (X_in + 1) / 2.0
  X_out = (X_out + 1) / 2.0

  # Plot real images
  for i in range(n_samples):
    pyplot.subplot(2, n_samples, 1 + i)
    pyplot.axis("off")
    pyplot.imshow(X_in[i])

  # Plot translated image
  for i in range(n_samples):
    pyplot.subplot(2, n_samples, 1 + n_samples + i)
    pyplot.axis("off")
    pyplot.imshow(X_out[i])

  # Save plot to file
  filename1 = "%s_generated_plot_%06d.png" % (name, (step+1))
  pyplot.savefig(filename1)
  pyplot.close()

In [ ]:
def train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA,
dataset):
  # Define properties of the training run
  n_epochs, n_batch, = 100, 1

  # Determine the output square shape of the discriminator
  n_patch = d_model_A.output_shape[1]

  # Unpack dataset
  trainA, trainB = dataset

  # Prepare image pool for fakes
  poolA, poolB = list(), list()

  # Calculate the number of batches per training epoch
  bat_per_epo = int(len(trainA) / n_batch)
  
  # Calculate the number of training iterations
  n_steps = bat_per_epo * n_epochs

  # Manually enumerate epochs
  for i in range(n_steps):

    # Select a batch of real samples
    X_realA, y_realA = generate_real_samples(trainA, n_batch, n_patch)
    X_realB, y_realB = generate_real_samples(trainB, n_batch, n_patch)

    # Generate a batch of fake samples
    X_fakeA, y_fakeA = generate_fake_samples(g_model_BtoA, X_realB, n_patch)
    X_fakeB, y_fakeB = generate_fake_samples(g_model_AtoB, X_realA, n_patch)

    # Update fakes from pool
    X_fakeA = update_image_pool(poolA, X_fakeA)
    X_fakeB = update_image_pool(poolB, X_fakeB)

    # Update generator B->A via adversarial and cycle loss
    g_loss2, _, _, _, _ = c_model_BtoA.train_on_batch([X_realB, X_realA], [y_realA,
    X_realA, X_realB, X_realA])

    # Update discriminator for A -> [real/fake]
    dA_loss1 = d_model_A.train_on_batch(X_realA, y_realA)
    dA_loss2 = d_model_A.train_on_batch(X_fakeA, y_fakeA)

    # Update generator A->B via adversarial and cycle loss
    g_loss1, _, _, _, _ = c_model_AtoB.train_on_batch([X_realA, X_realB], [y_realB,
    X_realB, X_realA, X_realB])

    # Update discriminator for B -> [real/fake]
    dB_loss1 = d_model_B.train_on_batch(X_realB, y_realB)
    dB_loss2 = d_model_B.train_on_batch(X_fakeB, y_fakeB)

    # Summarize performance
    print(">%d, dA[%.3f,%.3f] dB[%.3f,%.3f] g[%.3f,%.3f]" % (i+1, dA_loss1,dA_loss2,
    dB_loss1,dB_loss2, g_loss1,g_loss2))

    if (i+1) % (bat_per_epo * 1) == 0:
      # Plot A->B translation
      summarize_performance(i, g_model_AtoB, trainA, "AtoB")
      # Plot B->A translation
      summarize_performance(i, g_model_BtoA, trainB, "BtoA")

    if (i+1) % (bat_per_epo * 5) == 0:
      # Save the models
      save_models(i, g_model_AtoB, g_model_BtoA)

In [ ]:
# Load image data
dataset = load_real_samples("/content/drive/MyDrive/horse2zebra_256.npz")
print("Loaded", dataset[0].shape, dataset[1].shape)

# Define input shape based on the loaded dataset
image_shape = dataset[0].shape[1:]

# Generator: A -> B
g_model_AtoB = define_generator(image_shape)

# Generator: B -> A
g_model_BtoA = define_generator(image_shape)

# Discriminator: A -> [real/fake]
d_model_A = define_discriminator(image_shape)

# Discriminator: B -> [real/fake]
d_model_B = define_discriminator(image_shape)

# Composite: A -> B -> [real/fake, A]
c_model_AtoB = define_composite_model(g_model_AtoB, d_model_B, g_model_BtoA, image_shape)

# Composite: B -> A -> [real/fake, B]
c_model_BtoA = define_composite_model(g_model_BtoA, d_model_A, g_model_AtoB, image_shape)

# Train models
train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset)

Loaded (1187, 256, 256, 3) (1474, 256, 256, 3)
>1, dA[1.822,1.380] dB[0.749,1.900] g[15.962,19.376]
>2, dA[5.990,1.013] dB[1.555,3.308] g[17.078,21.629]
>3, dA[9.278,1.255] dB[1.956,4.311] g[26.415,20.179]
>4, dA[4.131,1.424] dB[1.871,2.025] g[18.368,16.921]
>5, dA[2.807,1.404] dB[4.583,3.977] g[22.963,16.790]
>6, dA[1.376,1.881] dB[1.984,4.846] g[18.818,15.191]
>7, dA[0.954,2.281] dB[1.202,4.281] g[18.498,17.238]
>8, dA[0.914,2.380] dB[0.983,1.934] g[15.318,13.745]
>9, dA[0.681,1.577] dB[0.924,0.854] g[19.459,18.350]
>10, dA[0.823,0.852] dB[0.948,1.018] g[16.164,15.422]
>11, dA[0.770,0.873] dB[1.299,1.250] g[16.740,13.756]
>12, dA[0.751,0.755] dB[2.083,1.424] g[16.584,15.715]
>13, dA[0.556,0.909] dB[1.150,0.641] g[13.333,14.131]
>14, dA[0.965,0.819] dB[1.114,1.183] g[17.376,17.489]
>15, dA[0.727,0.486] dB[0.787,1.523] g[13.389,12.552]
>16, dA[0.539,0.744] dB[0.634,1.405] g[12.635,11.548]
>17, dA[0.427,0.744] dB[0.636,0.792] g[9.707,10.363]
>18, dA[0.464,0.391] dB[0.687,0.563] g[12.343